# QFT Deployment on Neutral-Atom Systems
The Quantum Fourier Transform (QFT) deployment tutorial demonstrates how to implement a QFT circuit using the Bloqade framework, with special emphasis on hardware-specific optimizations for neutral-atom quantum computers. The tutorial begins by outlining the canonical QFT circuit—which is typically defined with Hadamard gates, controlled-phase rotations, and a cascade of controlled operations—and explains how this standard formulation is not immediately native to neutral-atom devices. This motivates the need for decomposing non-native operations into a sequence of hardware-compatible gates and reorganizing the circuit structure to reveal parallelism opportunities.

Central to the approach is the decomposition of the QFT circuit into neutral-atom native gates. Similar to the GHZ state preparation tutorial, non-native operations such as the CNOT are rewritten in terms of native operations. The tutorial leverages Bloqade’s extended QASM2 dialect by decorating functions with a custom compiler pass that uses Kirin’s rewriting tools. This pass automatically decomposes the circuit into the native gate set and optimizes the layout by grouping commuting operations together, thus exposing parallel execution paths. Barriers are strategically inserted to enforce constraints and guide the optimizer, ensuring that operations intended to be executed in parallel are properly aligned.

By the end of this tutorial you will be able to:
- Understand how to use parallelization on neutral atom computers
- Develop neutral atom optimized quantum algorithms through the bloqade framework
- Read a quantum computing circuit and be able to recognize the basic set of quantum gates needed to understand most quantum circuits readily available
- Exploit quantum advantage and integrate it in larger scale applications


This block will import all the required modules

In [1]:
from bloqade import qasm2
from kirin.dialects import ilist
import math
from bloqade.qasm2.emit import QASM2 # the QASM2 target
from bloqade.qasm2.parse import pprint # the QASM2 pretty printer
from bloqade.qasm2.rewrite.native_gates import RydbergGateSetRewriteRule
from kirin import ir
from kirin.rewrite import Walk
from bloqade.qasm2.passes import UOpToParallel, QASM2Fold

This block uses the dialect created in the GHZ tutorial

In [2]:
@ir.dialect_group(qasm2.extended)
def extended_opt(self):
    native_rewrite = Walk(RydbergGateSetRewriteRule(self)) # use Kirin's functionality to walk code line by line while applying neutral-atom gate decomposition as defined in Bloqade
    parallelize_pass = UOpToParallel(self) # review the code and apply parallelization using a heuristic
    agg_fold = QASM2Fold(self) # supports parallelization by unfolding loops to search for parallelization opportunities

    # here we define our new compiler pass
    def run_pass(
        kernel: ir.Method,
        *,
        fold: bool = True,
        typeinfer: bool = True,
        parallelize: bool = False,
    ):
        assert qasm2.extended.run_pass is not None
        qasm2.extended.run_pass(kernel, fold=fold, typeinfer=typeinfer) # apply the original run_pass to the lowered kernel
        native_rewrite.rewrite(kernel.code) # decompose all gates in the circuit to neutral atom gate set

        # here goes our parallelization optimizer; the order of the commands here matters!
        if parallelize:
            agg_fold.fixpoint(kernel)
            parallelize_pass(kernel)

    return run_pass

This block defines a function to build a Quantum Fourier Transform (QFT) circuit optimized for parallel execution. It uses a helper function to apply patterned layers of controlled-X gates and constructs the QFT with Hadamard gates, Rz rotations, and structured controlled operations.

The first section of the QFT circuit is canonically as follows, however, it is not in the native gate set for neutral-atom computers. Thus we decomposed the circuit into gates which are native to the architecture, and CNOTs that ressemble the GHZ problem which revealed oppurtunities for parallelization

We start with a sub-circuit of the QFT algorithm below.

<div align="center">
<picture>
   <img src="ImageYQuantumHBlock.png" style="width: 35vw; min-width: 330px;" >
</picture>
</div>

Next, we apply the circuit identity in fig 2(a) to get the following circuit.

<div align="center">
<picture>
   <img src="image copy 2.png" >
</picture>
</div>

<div align="center">
<picture>
   <img src="ImageYQuantumHBlockSecondExpantion.png" >
</picture>
</div>

Now we combine some Rz gates since they are diagonal and therefore commutable. This gives us the following circuit.

<div align="center">
<picture>
   <img src="Untitled-1.png" >
</picture>
</div>

By the identity in fig 2b, we can then decompose the circuit into the following:

<div align="center">
<picture>
   <img src="image copy 3.png" >
</picture>
</div>

<div align="center">
<picture>
   <img src="Untitled-2.png" >
</picture>
</div>

All circuit diagrams and decompositions are from [nature](https://www.nature.com/articles/s41598-023-35625-3#Fig5) and show the CNOTs ressembling the GHZ allowing us to take advantage of the parallelization proposed in the GHZ tutorial provided.

<div align="center">
<picture>
   <img src="GHZ_linear.png" style="width: 35vw; min-width: 330px;" >
</picture>
</div>

<div align="center">
<picture>
   <img src="GHZ_parallel.png" style="width: 25vw; height: 25vw;" >
</picture>
</div>

In [3]:
import qiskit.qasm2 as qq
import random

@extended_opt
def dyadic_roatation_layer(qreg : qasm2.QReg, n : int, start : int):
    # Apply a sequence of dyadic-like rotations
    for j in range(start + 1, n):
        qasm2.rz(qreg[j], math.pi / (2**(j + 2)))

@extended_opt
def cnot_fanout_layer(qreg : qasm2.QReg, n : int, ctrl : int):
    # Apply the CNOT fanout in its equivalent sequential form
    for j in range(ctrl + 1, n):
        qasm2.cx(qreg[ctrl], qreg[j])

def qft(n: int, bit_str : str, parallelize: bool = True):
    @extended_opt(parallelize=parallelize)
    def qft_program():
        qreg = qasm2.qreg(n)
        creg = qasm2.creg(n)

        # Initialize the qubits to the state in the given bit-string
        for i in range(len(bit_str)):
            if bit_str[i] == '1':
                qasm2.x(qreg[i])

        for i in range(n):
            qasm2.h(qreg[i])

            dyadic_roatation_layer(qreg, n, i)

            cnot_fanout_layer(qreg, n, i)

            dyadic_roatation_layer(qreg, n, i)

            cnot_fanout_layer(qreg, n, i)

            # Apply the last rotation
            qasm2.rz(qreg[i], ((2**(n - i + 1) - 1) * math.pi) / (2**(n - i + 2)))
        for i in range(n):
            qasm2.measure(qreg[i],creg[i])

        return creg # return register for simulation

    return qft_program

target = QASM2()
ast = target.emit(qft(4, "0101"))
pprint(ast)

/Users/Plopes/Documents/GitHub/.venv/lib/python3.13/site-packages/cirq/circuits/circuit_operation.py:173: FutureWarning: In cirq 1.6 the default value of `use_repetition_ids` will change to
`use_repetition_ids=False`. To make this warning go away, please pass
explicit `use_repetition_ids`, e.g., to preserve current behavior, use

  CircuitOperations(..., use_repetition_ids=True)
  warnings.warn(msg, FutureWarning)
/Users/Plopes/Documents/GitHub/.venv/lib/python3.13/site-packages/cirq/circuits/circuit_operation.py:173: FutureWarning: In cirq 1.6 the default value of `use_repetition_ids` will change to
`use_repetition_ids=False`. To make this warning go away, please pass
explicit `use_repetition_ids`, e.g., to preserve current behavior, use

  CircuitOperations(..., use_repetition_ids=True)
  warnings.warn(msg, FutureWarning)
/Users/Plopes/Documents/GitHub/.venv/lib/python3.13/site-packages/cirq/circuits/circuit_operation.py:173: FutureWarning: In cirq 1.6 the default value of `use_repet

OPENQASM 2.0;
include "qelib1.inc";
qreg qreg[4];
creg creg[4];
U(3.14159265358979, 0.0, 3.14159265358979) qreg[3];
U(3.14159265358979, 0.0, 3.14159265358979) qreg[1];
U(1.5707963267949, 0.0, 3.14159265358979) qreg[0];
U(0.0, 0.0, 0.39269908169872414) qreg[1];
U(0.0, 0.0, 0.19634954084936207) qreg[2];
U(0.0, 0.0, 0.09817477042468103) qreg[3];
U(1.5707963267949, 3.14159265358979, 3.14159265358979) qreg[3];
U(1.5707963267949, 3.14159265358979, 3.14159265358979) qreg[1];
cz qreg[0], qreg[1];
U(1.5707963267949, 0.0, 0.0) qreg[1];
U(1.5707963267949, 3.14159265358979, 3.14159265358979) qreg[2];
cz qreg[0], qreg[2];
U(1.5707963267949, 0.0, 0.0) qreg[2];
cz qreg[0], qreg[3];
U(1.5707963267949, 0.0, 0.0) qreg[3];
U(0.0, 0.0, 0.39269908169872414) qreg[1];
U(0.0, 0.0, 0.19634954084936207) qreg[2];
U(0.0, 0.0, 0.09817477042468103) qreg[3];
U(1.5707963267949, 3.14159265358979, 3.14159265358979) qreg[1];
cz qreg[0], qreg[1];
U(1.5707963267949, 0.0, 0.0) qreg[1];
U(1.5707963267949, 3.14159265358979, 

This block tests the QFT recording measurements over a large set of experiments. Since QFT only encodes a phase into the states, under measurement the output distribution should be uniform regardless of input. A random bit string is inputted at the start of the QFT for this reason

In [ ]:
from bloqade.pyqrack import PyQrack
from collections import Counter
import random

device = PyQrack(dynamic_qubits=True, pyqrack_options={"isBinaryDecisionTree": False})

bit_str = ""
for i in range(4):
    bit_str += str(random.randint(0,1))

kernel = qft(4, bit_str)
results = device.multi_run(kernel, _shots=10000)

def to_bitstrings(results):
    return Counter(map(lambda result:"".join(map(str, result)), results))

counts = to_bitstrings(results)

for key, value in counts.items():
    print(key, value)

One area of the decomposed Quantum Fourier Transform (QFT) that lacks clear route of optimization is the sequence of controlled rotations. A promising direction could be the design of a more complex global gate that combines multiple rotations simultaneously requiring a physics/engineering effort. Since global operations may introduce significantly less error than local operations and there would be 1 operation vs. n operations, this could yield orders-of-magnitude improvements in speed. This could be a worthwile endeavour despite the engineering difficulty given how common the QFT is in algorithms.

Additionally, because the QFT rotations scale as π/2ᵏ, a cutoff threshold can be introduced below which rotations are omitted, as their impact on the state is negligible—further accelerating computation with minimal loss in accuracy. 

Another potential optimization involves using additional ancilla qubits to replicate shared control lines. This could reduce circuit depth from linear to logarithmic. For example, this is 3 CNOT gates that share a control line:
<div align="center">
<picture>
   <img src="image.png" style="width: 35vw; min-width: 330px;" >
</picture>
</div>

In this second image, these CNOT gates can be parallelized as by 'copying' the control qubit onto the additional 4th qubit, a selection of CNOTS can control off the additional qubit

<div align="center">
<picture>
   <img src="image copy.png" style="width: 35vw; min-width: 330px;" >
</picture>
</div>


This is left as an exercise to implement for the QFT but it would yield a log improvement in depth whilst adding linearly in the amount of qubits